# RECEIVING DATA FROM NOAA SATTELITE

So, I would like to receive data from the NOAA satellite. Typically the ranges of the noaa sattelites vary depending on the sat, but almost all use an FM modulation. 
    - NOAA 15: 1345-1357 MHz but has APT service that runs at 137 MHz to 138 MHz
    - NOAA 18: 137.0-137.3 MHz
    - Newer sats operate at up to 191.31 GHz

These sattelites are used for weather forecasting and are also used for amateur radio. The data is transmitted using the APT (Automatic Picture Transmission) protocol. So I am going to try to receive data and load it into the 


I am going to be using SoapySDR which has the simplesoapy library. I am trying to use the simplesoapy library to create a simple audio stream by demodulating an FM radio signal using the hackrf.

In [13]:
import numpy as np
import simplesoapy
import matplotlib.pyplot as plt
from scipy.signal import decimate, butter, lfilter
import sounddevice as sd

# Set up HackRF
sdr = simplesoapy.SoapyDevice("device=hackrf")
sdr.sample_rate = 2.048e6  # 2.048 MHz sample rate
sdr.frequency = 101.1e6    # Frequency of the FM station (change to your target station)
sdr.gain = 20              # Adjust gain as needed

# Capture samples
buffer_size = 2048
samples = sdr.read_samples(buffer_size)

# FM Demodulation function
def fm_demodulate(samples, sample_rate, decimation_factor):
    samples = samples - np.mean(samples)  # Center the signal
    angle_diff = np.angle(samples[1:] * np.conj(samples[:-1]))  # Derivative to get phase change
    b, a = butter(5, 0.01)  # Low-pass filter
    audio = lfilter(b, a, angle_diff)  # Filter the demodulated signal
    audio = decimate(audio, decimation_factor)  # Downsample
    return audio

# Process FM demodulation
audio = fm_demodulate(samples, sample_rate=2.048e6, decimation_factor=10)

# Plotting the audio output
plt.plot(audio[:1000])  # Plotting first 1000 samples of audio
plt.title("Demodulated Audio Signal")
plt.xlabel("Sample Number")
plt.ylabel("Amplitude")
plt.show()

# Playing audio
sd.play(audio, samplerate=204800)  # Adjust sample rate as needed


[INFO] Opening HackRF One #0 66a062dc2272319f...
Sample rate 2048000.0 Hz is not supported, setting it to 2000000.0 Hz!


AttributeError: 'SoapyDevice' object has no attribute 'read_samples'